In [1]:
import sys
import os

# Add the parent directory to the Python path to get optimize module
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../')))

import nest_asyncio

# Apply the nest_asyncio patch: let's us run async code in Jupyter
nest_asyncio.apply()

In [ ]:
ID_FIELD_NAME = "chunk_id"
CHUNK_FIELD_NAME = "content"

def parse_index_items(items, storage_type="hash"):
    if storage_type == "json":
        return [
            {"item_id": item[ID_FIELD_NAME], "text": item[CHUNK_FIELD_NAME]}
            for item in items
        ]
    else:
        return [
            {
                "item_id": item[ID_FIELD_NAME.encode()].decode(),
                "text": item[CHUNK_FIELD_NAME.encode()].decode(),
            }
            for item in items
        ]


def get_items_by_pattern(client, pattern, storage_type="hash"):
    cursor = "0"
    items = []

    while cursor != 0:
        cursor, keys = client.scan(cursor=cursor, match=pattern)
        for key in keys:
            if storage_type == "json":
                items.append(client.json().get(key))
            else:
                items.append(client.hgetall(key))

    return items

In [ ]:
from redis import Redis

client = Redis.from_url("redis://localhost:6379/0")
items = get_items_by_pattern(client, "mazda:*", storage_type="hash")